In [1]:
import geopandas as gpd
from folium import (
    LayerControl,
    Element,
    TileLayer,
)
import plotly.express as px




from core.downloads.geosampa import get_capabilities, get_features

# Promoção da Sustentabilidade Ambiental, Gestão de risco

## Formulário 7

O formulário associado a este notebook solicita os dados sobre 
`áreas de risco (hidrológico e deslizamentos)`. A justificativa para estes dados é a seguinte:

> As áreas de risco estão relacionadas à construção de moradias, em sua maioria em condições precárias, em locais com geológico-geotécnicas frágeis, não recomendadas para ocupação. O impacto de chuvas concentradas intensas, características de eventos climáticos extremos deve ser monitorado pelo Município, através de políticas públicas de gestão de risco e promoção da resiliência climática.

In [ ]:
get_capabilities('hidrológico')

In [ ]:
get_capabilities('deslizamento')

Como o formulário cita apenas a camada `proteção e defesa civil/mapeamento de areas de risco`, assumirei que a referência a deslizamentos seja sobre a camada de risco geológico.

In [ ]:
df_hid = get_features('geoportal:risco_hidrologico')
df_hid.head()

In [ ]:
df_geo = get_features('geoportal:area_risco_geologico')
df_geo.head()

Além dos dados de riscos hidrogeológicos, também precisaremos dos dados do Censo de 2022 para a estimativa populacional. Os dados básicos, como número de domicílios e população, são disponibilizados diretamente no geopackage com as geometrias de setores censitários.

In [ ]:
df_censo = gpd.read_file('https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/setores/gpkg/UF/SP/SP_setores_CD2022.gpkg')
df_censo.head()

In [ ]:
df_censo = df_censo.loc[df_censo['CD_MUN']=='3550308']
df_censo.head()

## Ajustando as projeções

Vamos revisar os sistemas de coordenadas de todos os geodataframes para garantir que estão na mesma projeção.

In [ ]:
for gdf in [df_geo, df_hid, df_censo]:
    print(gdf.columns[:5])
    print(gdf.crs)

Como o geodataframe do censo está em outro crs, precisamos convertê-lo para o `epsg:31983`.

In [9]:
df_censo = df_censo.to_crs('EPSG:31983')

# Calculando a população de cada área de risco

Primeiro, vamos inspecionar visualmente os geodataframes.

In [ ]:
df_geo.explore()

In [ ]:
df_censo.iloc[:500].explore()

Depois, precisaremos calcular a interseção de cada um dos dataframes de risco com os setores censitários.

In [ ]:
ol1 = gpd.overlay(df_geo, df_censo,
            how='intersection',
            keep_geom_type=True)
ol1.head()

Vamos avaliar visualmente o resultado da interseção com base na primeira área de risco.

In [ ]:
id_area = ol1.loc[:, 'id'].iloc[0]
id_area

In [ ]:
cd_setor_list = ol1.loc[ol1['id']==id_area, 'CD_SETOR'].tolist()
ol1.loc[ol1['id']=='area_risco_geologico.1', ['id', 'CD_SETOR']]

In [ ]:
m = df_censo[df_censo['CD_SETOR'].isin(
    cd_setor_list)].explore(name='setor censitário')

m = df_geo[df_geo['id'] == 'area_risco_geologico.1'].explore(
    m=m, color='orange', name='area de risco')

filtered_ol1 = ol1.loc[ol1['id'] == 'area_risco_geologico.1', [
    'id', 'CD_SETOR', 'geometry']]
m = filtered_ol1.explore(m=m, color='purple', name='interseção')

LayerControl().add_to(m)

m

Na inspeção visual, nota-se que algumas interseções não aparentam representar áreas com moradias, mas simplesmente leves discrepâncias no desenho dos polígonos sobre áreas não populadas (ruas,  canteiros, etc.).

Por isso, precisaremos limpar essas interseções da nossa base. Podemos utilizar um buffer negativo nos polígonos, de modo que polígonos com altura/largura menor do que a metade do valor aplicado no buffer se tornarão vazios.

In [ ]:
# Assumindo que um terreno com 8m ou menos de largura não possui uma casa
buffer = -1*(8/2)

ol2 = ol1.copy()
ol2['debuffed'] = ol2.buffer(buffer)
ol2 = ol2[~ol2['debuffed'].is_empty]
ol2.head()

In [ ]:
m = df_censo[df_censo['CD_SETOR'].isin(
    cd_setor_list)].explore(name='setor censitário')

m = df_geo[df_geo['id'] == 'area_risco_geologico.1'].explore(
    m=m, color='orange', name='area de risco')

filtered_ol2 = ol2.loc[ol2['id'] == 'area_risco_geologico.1', [
    'id', 'CD_SETOR', 'geometry']]
m = filtered_ol2.explore(m=m, color='purple', name='interseção')

LayerControl().add_to(m)

m

Vimos que as inteseções indesejadas sumiram do resultado, então podemos continuar os cálculos.

O primeiro passo é calcular a proporção entre a área da interseção e a área do setor censitário.

In [ ]:
ol2['inter_area'] = ol2['geometry'].area
ol2.head()

In [ ]:
df_censo_area = df_censo[['CD_SETOR', 'geometry']]
df_censo_area['area_setor'] = df_censo_area['geometry'].area
df_censo_area

In [ ]:
ol2 = ol2.merge(df_censo_area[['CD_SETOR', 'area_setor']], how='left',
          on='CD_SETOR')
ol2.head()

In [ ]:
ol2['prop_setor'] = ol2['inter_area']/ol2['area_setor']
ol2.head()

Sabendo que a proporção da interseção não deve ser maior do que 1, vamos conferir os valores de interseção.

In [ ]:
ol2['prop_setor'].hist()

Como a distribuição apresenta muitos valores abaixo de 0.2, vamos analisar esses casos visualmente para confirmar que fazem sentido.

In [23]:
filtro_intersecoes_pequenas = ol2['prop_setor']<=0.2
setores_02 = ol2.loc[filtro_intersecoes_pequenas, 'CD_SETOR'].tolist()
areas_risco_02 = ol2.loc[filtro_intersecoes_pequenas, 'id'].tolist()

In [ ]:
# Criar o mapa com as camadas
m = df_censo[df_censo['CD_SETOR'].isin(setores_02)].explore(name='Setor Censitário')

m = df_geo[df_geo['id'].isin(areas_risco_02)].explore(
    m=m, color='orange', name='Área de Risco')

filtered_ol2 = ol2.loc[filtro_intersecoes_pequenas, ['id', 'CD_SETOR', 'geometry', 'prop_setor']]
m = filtered_ol2.explore(m=m, color='purple', name='Interseção')

LayerControl().add_to(m)

# Adicionar a legenda personalizada
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 120px; 
            background-color: white; z-index:1000; padding: 10px; 
            border: 2px solid grey; border-radius: 5px;">
    <h4>Legenda</h4>
    <i style="background: orange; width: 10px; height: 10px; display: inline-block;"></i> Área de Risco<br>
    <i style="background: purple; width: 10px; height: 10px; display: inline-block;"></i> Interseção<br>
    <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Setor Censitário
</div>
"""
m.get_root().html.add_child(Element(legend_html))

m

A inspeção visual mostra que as interseções aparentam ser significativas, mas se tratam de interseções pequenas em comparação com os setores censitários, ocorrendo principalmente em setores sensitários maiores. Portanto, a eliminação das interseções e o cálculo da proporção da população parece ter funcionado corretamente.

Porém, um ponto notado durante a inspeção visual e oportuno de destaque é um possível viés de superestimação em áreas pouco povoadas, como no entorno de parques. Por se tratarem de áreas de baixa densidade demográfica, estão mais sujeitas a estarem incluídas em setores censitários maiores e a uma distribuição heterogênea da população dentro do seu território. Um exemplo desse caso pode ser visto nas áreas de risco de cd_identificador 1227 a 1230.

In [ ]:
ol2['pop_estimada'] = ol2['v0001']*ol2['prop_setor']
ol2[['id','CD_SETOR','v0001', 'prop_setor', 'pop_estimada']].sort_values('pop_estimada', ascending=False)

In [ ]:
ol2.loc[:, ['id', 'pop_estimada']].groupby('id').sum()

In [ ]:
area_risco_pop = (
    ol2
    .loc[:, ['id', 'pop_estimada']]
    .groupby('id')
    .sum()
    .round(0)
    .reset_index()
)
area_risco_pop['pop_estimada'] = area_risco_pop['pop_estimada'].astype(int)
area_risco_pop

In [ ]:
df_geo_pop = df_geo.merge(
    area_risco_pop,
    how='left'
)
df_geo_pop

In [ ]:
df_tree = df_geo_pop.groupby(['tx_tipo_processo_geologico', 'tx_grau_de_risco_geologico'], as_index=False)['pop_estimada'].sum()
df_tree

In [ ]:
# Criar o gráfico do tipo treeview
fig = px.treemap(
    df_tree,
    path=['tx_grau_de_risco_geologico', 'tx_tipo_processo_geologico'],  # Caminho hierárquico
    values='pop_estimada',               # Valores para o tamanho das caixas
    title='Somatório de População Estimada por Tipo de Processo Geológico'
)

# Exibir o gráfico
fig.show()

In [ ]:
# Criar o mapa base
m = df_geo_pop.explore(
    column='pop_estimada',  # Coluna para o mapa coroplético
    cmap='YlOrRd',          # Paleta de cores
    legend=True,            # Adicionar legenda
    tooltip=['tx_grau_de_risco_geologico', 'tx_tipo_processo_geologico', 'pop_estimada'],  # Informações no tooltip
    tiles='CartoDB positron',  # Mapa base
    name='Áreas de Risco'   # Nome da camada
)

# Filtrar os setores censitários presentes em ol2
setores_presentes = df_censo[df_censo['CD_SETOR'].isin(ol2['CD_SETOR'].unique())]

# Adicionar os setores censitários ao mapa
m = setores_presentes.explore(
    m=m,
    color='blue',
    name='Setores Censitários'
)

# Adicionar outra camada base (Google Satellite)
TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite'
).add_to(m)

# Adicionar controle de camadas
LayerControl(position='bottomleft').add_to(m)

# Exibir o mapa
m


In [ ]:
area_risco_pop.shape